In [3]:
%matplotlib inline
import torch
import numpy as np
import matplotlib.pylab as plt
import sys
import d2l.torch as d2l

1.获取与读取数据

In [ ]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

num_inputs,num_outputs,num_hiddens = 784, 10, 256
W1 = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_hiddens)), dtype=torch.float)
b1 = torch.zeros(num_hiddens, dtype=torch.float)
W2 = torch.tensor(np.random.normal(0, 0.01, (num_hiddens, num_outputs)), dtype=torch.float)
b2 = torch.zeros(num_outputs, dtype=torch.float)
params = [W1, b1, W2, b2]
for param in params:
    param.requires_grad_(True)

2.定义激活函数 && 定义模型 && 训练模型

In [9]:
def relu(X):
    return torch.max(input=X, other=torch.tensor(0.0))
#定义模型
def net(X):
    X = X.view((-1, num_inputs))  # 展平
    H = relu(torch.matmul(X, W1) + b1)  # 隐藏层
    return torch.matmul(H, W2) + b2  # 输出层
#定义损失函数
loss = torch.nn.CrossEntropyLoss()
#训练模型
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
    return acc_sum / n

def sgd(params, lr, batch_size):
    """小批量随机梯度下降算法"""
    if lr is None or batch_size is None:
        raise ValueError("学习率(lr)和批量大小(batch_size)不能为None")
    with torch.no_grad():
        for param in params:
            if param.grad is not None:
                param -= lr * param.grad / batch_size

def train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params=None,lr=None,optimizer=None):
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
        for X,y in train_iter:
            y_hat = net(X)
            l = loss(y_hat,y).sum()
            # 梯度清零
            if optimizer is not None:
                optimizer.zero_grad()
            elif params is not None and params[0].grad is not None:
                for param in params:
                    param.grad.data.zero_()
            l.backward()
            if optimizer is None:
                sgd(params,lr,batch_size)
            else:
                optimizer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
            n += y.shape[0]
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'% (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))
num_epochs, lr = 5, 100.0
train_ch3(net, train_iter, test_iter, loss, num_epochs,batch_size, params, lr)

epoch 1, loss 0.0031, train acc 0.709, test acc 0.742
epoch 2, loss 0.0019, train acc 0.822, test acc 0.839
epoch 3, loss 0.0017, train acc 0.844, test acc 0.835
epoch 4, loss 0.0015, train acc 0.855, test acc 0.832
epoch 5, loss 0.0015, train acc 0.863, test acc 0.842
